# Google Drive 마이그레이션 테스트 (간단 버전)

Google Colab의 내장 인증을 사용합니다.

## 사용 방법
1. 셀을 순서대로 실행하세요
2. Google 계정으로 로그인하세요
3. 파일 목록을 확인하세요


In [ ]:
# 런타임 설정 및 라이브러리 설치
import sys
import os

# Python 버전 확인
print(f"Python 버전: {sys.version}")
print(f"Python 실행 경로: {sys.executable}")

# 필요한 라이브러리 설치
%pip install google-api-python-client pandas tqdm
import pandas as pd
from tqdm import tqdm
from datetime import datetime

# Google Colab 내장 인증 사용
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

print("✅ 라이브러리 로드 완료")


✅ 라이브러리 로드 완료


In [ ]:
# 이중 계정 인증 시스템
class DualAccountManager:
    def __init__(self):
        self.personal_service = None
        self.work_service = None
        self.personal_creds = None
        self.work_creds = None
        self.work_domain = "@ua-all.co.kr"  # 회사 계정 도메인 고정
    
    def authenticate_personal(self):
        """개인 계정 인증"""
        print("🔐 개인 Google 계정 인증을 시작합니다...")
        print("⚠️  개인 계정은 회사 도메인이 아닌 계정을 사용해야 합니다.")
        print("팝업 창에서 개인 Google 계정으로 로그인하세요.")
        
        # Google Colab 내장 인증 사용
        auth.authenticate_user()
        
        # Google Drive API 서비스 생성
        self.personal_service = build('drive', 'v3')
        
        # 사용자 정보 확인 및 도메인 검증
        try:
            about = self.personal_service.about().get(fields='user').execute()
            user_info = about.get('user', {})
            email = user_info.get('emailAddress', '')
            
            # 개인 계정 도메인 검증 (회사 도메인이 아닌지 확인)
            if email.endswith(self.work_domain):
                print(f"❌ 잘못된 개인 계정입니다!")
                print(f"   입력된 이메일: {email}")
                print(f"   개인 계정은 회사 도메인({self.work_domain})이 아닌 계정을 사용해야 합니다.")
                print(f"   올바른 개인 계정으로 다시 로그인해주세요.")
                self.personal_service = None
                return False
            
            print(f"✅ 개인 계정 인증 완료!")
            print(f"   - 이름: {user_info.get('displayName', 'Unknown')}")
            print(f"   - 이메일: {email}")
            print(f"   - 도메인 검증: ✅ 통과 (회사 도메인 아님)")
            return True
        except Exception as e:
            print(f"❌ 개인 계정 인증 실패: {e}")
            return False
    
    def authenticate_work(self):
        """회사 계정 인증"""
        print("\n🔐 회사 Google 계정 인증을 시작합니다...")
        print(f"⚠️  회사 계정은 {self.work_domain} 도메인을 사용해야 합니다.")
        print("팝업 창에서 회사 Google 계정으로 로그인하세요.")
        
        # Google Colab 내장 인증 사용 (새로운 세션)
        auth.authenticate_user()
        
        # Google Drive API 서비스 생성
        self.work_service = build('drive', 'v3')
        
        # 사용자 정보 확인 및 도메인 검증
        try:
            about = self.work_service.about().get(fields='user').execute()
            user_info = about.get('user', {})
            email = user_info.get('emailAddress', '')
            
            # 도메인 검증
            if not email.endswith(self.work_domain):
                print(f"❌ 잘못된 회사 계정입니다!")
                print(f"   입력된 이메일: {email}")
                print(f"   필요한 도메인: {self.work_domain}")
                print(f"   올바른 회사 계정으로 다시 로그인해주세요.")
                self.work_service = None
                return False
            
            print(f"✅ 회사 계정 인증 완료!")
            print(f"   - 이름: {user_info.get('displayName', 'Unknown')}")
            print(f"   - 이메일: {email}")
            print(f"   - 도메인 검증: ✅ 통과")
            return True
        except Exception as e:
            print(f"❌ 회사 계정 인증 실패: {e}")
            return False
    
    def force_reauth_work(self):
        """회사 계정 강제 재인증"""
        print("\n🔄 회사 계정 강제 재인증을 시작합니다...")
        print("⚠️  이전 인증을 무시하고 새로운 회사 계정으로 로그인합니다.")
        print(f"⚠️  회사 계정은 {self.work_domain} 도메인을 사용해야 합니다.")
        
        # 기존 서비스 초기화
        self.work_service = None
        
        # Google Colab 내장 인증 사용 (새로운 세션)
        auth.authenticate_user()
        
        # Google Drive API 서비스 생성
        self.work_service = build('drive', 'v3')
        
        # 사용자 정보 확인 및 도메인 검증
        try:
            about = self.work_service.about().get(fields='user').execute()
            user_info = about.get('user', {})
            email = user_info.get('emailAddress', '')
            
            # 도메인 검증
            if not email.endswith(self.work_domain):
                print(f"❌ 잘못된 회사 계정입니다!")
                print(f"   입력된 이메일: {email}")
                print(f"   필요한 도메인: {self.work_domain}")
                print(f"   올바른 회사 계정으로 다시 로그인해주세요.")
                self.work_service = None
                return False
            
            print(f"✅ 회사 계정 인증 완료!")
            print(f"   - 이름: {user_info.get('displayName', 'Unknown')}")
            print(f"   - 이메일: {email}")
            print(f"   - 도메인 검증: ✅ 통과")
            return True
        except Exception as e:
            print(f"❌ 회사 계정 인증 실패: {e}")
            return False
    
    def get_account_info(self, account_type):
        """계정 정보 조회"""
        service = self.personal_service if account_type == 'personal' else self.work_service
        
        if not service:
            print(f"❌ {account_type} 계정이 인증되지 않았습니다.")
            return None
        
        try:
            about = service.about().get(fields='user,storageQuota').execute()
            user_info = about.get('user', {})
            storage_quota = about.get('storageQuota', {})
            
            return {
                'user': user_info,
                'storage': storage_quota
            }
        except Exception as e:
            print(f"❌ {account_type} 계정 정보 조회 실패: {e}")
            return None

# 이중 계정 관리자 인스턴스 생성
account_manager = DualAccountManager()

print("✅ 이중 계정 인증 시스템 준비 완료!")


🔐 Google 계정 인증을 시작합니다...
팝업 창에서 Google 계정으로 로그인하세요.
✅ 인증 완료!


In [ ]:
# 1단계: 개인 계정 인증
print("=" * 60)
print(" 1단계: 개인 계정 인증")
print("=" * 60)

# 개인 계정 인증 실행
personal_auth_success = account_manager.authenticate_personal()

if personal_auth_success:
    # 개인 계정 정보 조회
    personal_info = account_manager.get_account_info('personal')
    if personal_info:
        user_info = personal_info['user']
        storage_info = personal_info['storage']
        
        print(f"\n📊 개인 계정 상세 정보:")
        print(f"  - 이름: {user_info.get('displayName', 'Unknown')}")
        print(f"  - 이메일: {user_info.get('emailAddress', 'Unknown')}")
        
        if storage_info:
            total_space = int(storage_info.get('limit', 0))
            used_space = int(storage_info.get('usage', 0))
            free_space = total_space - used_space
            
            def format_bytes(bytes_value):
                if bytes_value == 0:
                    return "0B"
                size_names = ["B", "KB", "MB", "GB", "TB"]
                i = 0
                while bytes_value >= 1024 and i < len(size_names) - 1:
                    bytes_value /= 1024.0
                    i += 1
                return f"{bytes_value:.1f}{size_names[i]}"
            
            print(f"  - 총 용량: {format_bytes(total_space)}")
            print(f"  - 사용량: {format_bytes(used_space)}")
            print(f"  - 여유 공간: {format_bytes(free_space)}")
else:
    print("❌ 개인 계정 인증에 실패했습니다. 다시 시도해주세요.")


👤 사용자 정보 조회 중...
✅ 사용자 정보:
  - 이름: 김태형
  - 이메일: thkim@us-all.co.kr


In [ ]:
# 3단계: 인증 상태 확인
print("\n" + "=" * 60)
print(" 3단계: 인증 상태 확인")
print("=" * 60)

# 인증 상태 확인
personal_ready = account_manager.personal_service is not None
work_ready = account_manager.work_service is not None

print(f"🔐 인증 상태:")
print(f"  - 개인 계정: {'✅ 인증됨' if personal_ready else '❌ 미인증'}")
print(f"  - 회사 계정: {'✅ 인증됨' if work_ready else '❌ 미인증'}")

if personal_ready and work_ready:
    print(f"\n🎉 모든 계정 인증 완료!")
    print(f"   이제 마이그레이션을 시작할 수 있습니다.")
    
    # 계정 정보 요약
    print(f"\n📋 계정 정보 요약:")
    personal_info = account_manager.get_account_info('personal')
    work_info = account_manager.get_account_info('work')
    
    if personal_info and work_info:
        personal_email = personal_info['user'].get('emailAddress', 'Unknown')
        work_email = work_info['user'].get('emailAddress', 'Unknown')
        
        print(f"  - 소스 계정 (개인): {personal_email}")
        print(f"  - 대상 계정 (회사): {work_email}")
        
        # 용량 비교
        personal_storage = personal_info['storage']
        work_storage = work_info['storage']
        
        if personal_storage and work_storage:
            personal_used = int(personal_storage.get('usage', 0))
            work_free = int(work_storage.get('limit', 0)) - int(work_storage.get('usage', 0))
            
            print(f"\n💾 용량 분석:")
            print(f"  - 개인 계정 사용량: {format_bytes(personal_used)}")
            print(f"  - 회사 계정 여유 공간: {format_bytes(work_free)}")
            
            if personal_used > work_free:
                print(f"  ⚠️  경고: 개인 계정 사용량이 회사 계정 여유 공간보다 큽니다!")
                print(f"      마이그레이션 전에 용량을 확인하세요.")
            else:
                print(f"  ✅ 회사 계정에 충분한 여유 공간이 있습니다.")
else:
    print(f"\n❌ 모든 계정을 인증해야 마이그레이션을 진행할 수 있습니다.")
    print(f"   위의 인증 단계를 다시 실행해주세요.")


In [ ]:
# 회사 계정 강제 재인증 (필요시 실행)
print("=" * 60)
print(" 회사 계정 강제 재인증")
print("=" * 60)
print("⚠️  이 셀은 회사 계정 인증이 실패했을 때만 실행하세요.")
print("⚠️  실행 전에 브라우저에서 Google 계정을 완전히 로그아웃하세요.")
print("⚠️  그 후 올바른 회사 계정(@ua-all.co.kr)으로 로그인하세요.")
print("\n실행하려면 아래 코드의 주석을 해제하세요:")
print("# work_reauth_success = account_manager.force_reauth_work()")

# 주석 해제하여 실행
# work_reauth_success = account_manager.force_reauth_work()


In [ ]:
# 2단계: 회사 계정 인증
print("\n" + "=" * 60)
print(" 2단계: 회사 계정 인증")
print("=" * 60)

# 회사 계정 인증 실행
work_auth_success = account_manager.authenticate_work()

if work_auth_success:
    # 회사 계정 정보 조회
    work_info = account_manager.get_account_info('work')
    if work_info:
        user_info = work_info['user']
        storage_info = work_info['storage']
        
        print(f"\n📊 회사 계정 상세 정보:")
        print(f"  - 이름: {user_info.get('displayName', 'Unknown')}")
        print(f"  - 이메일: {user_info.get('emailAddress', 'Unknown')}")
        
        if storage_info:
            total_space = int(storage_info.get('limit', 0))
            used_space = int(storage_info.get('usage', 0))
            free_space = total_space - used_space
            
            print(f"  - 총 용량: {format_bytes(total_space)}")
            print(f"  - 사용량: {format_bytes(used_space)}")
            print(f"  - 여유 공간: {format_bytes(free_space)}")
else:
    print("❌ 회사 계정 인증에 실패했습니다. 다시 시도해주세요.")


📁 파일 목록 조회 중...
✅ 파일 목록 (최대 10개):
   1. 교육과정별 판매 상세 데이터(25년) (987.0KB) 🔗
      ID: 1Xt2poreEJcKSzahZVa0AcgJfbbf1w8U_DCJAMIlhzO0
      타입: application/vnd.google-apps.spreadsheet
      수정일: 2025-09-05T04:32:59.906Z

   2. (IVE)어스캠퍼스 데일리 리포트 (98.8KB) 🔗
      ID: 10x30Gw5ho-4fm-BPrcv803Y9Ww1aUYwPqBKn5tSzTQ4
      타입: application/vnd.google-apps.spreadsheet
      수정일: 2025-09-05T01:30:04.180Z

   3. 어스플러스_운영지표 (41.7KB) 🔗
      ID: 1edcFkQGvcXVhkMhI2WNwd9nMfxE1gkp4eizHIHYYC8s
      타입: application/vnd.google-apps.spreadsheet
      수정일: 2025-09-05T00:20:36.211Z

   4. 퀀팃과 스마일게이트자산운용 분석
 (18.0KB) 🔗
      ID: 1J4NfUSqglWoy-SQdfYU93Z-8nYAd81dQaBRu7Uj37r4
      타입: application/vnd.google-apps.document
      수정일: 2025-09-04T04:19:39.360Z

   5. US-23826-2.csv (1599.9KB) 🔒
      ID: 1oEjdWP3L52X7JMu_RY3aCiYmonJe5oTY
      타입: text/csv
      수정일: 2025-09-03T09:30:26.786Z

   6. bq-results-20250903-092927-1756891778011 (폴더) 🔒
      ID: 1KY283XKvP7XDqTdJ1RIAIAFZo8N5Hvgz
      타입: application/vnd.goog